In [572]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import warnings
warnings.simplefilter('ignore')
import mpmath
from mpmath import *
from mpmath import mp 

* ### Аналитическое решение

In [573]:
def analytic_solution(x,t):
    return mp.cos(x+mpf(2.0*t))

In [574]:
def analytic_points(t):
        x1 = np.linspace(0,1,11)
        u = []
        for x in x1:
                res = analytic_solution(x,t)
                u.append(float(res))
        return x1,u

* ### Численное решение

In [575]:
def solve(koef,kurant):
    solutions = []; k = 1/kurant
    L0 = 11

    L = koef * (L0 - 1) + 1
    # T = int((L-1)/(kurant))
    h = mpf(str(1.0/(L-1))); 
    T = int((L-1)*(1/kurant)+1)
    tau = mpf(str(1.0/(T-1)))
    # x, h = np.linspace(0, 1, L, retstep=True)
    x = np.arange(0,1 + h,h)
    # print(len(x))
    t = np.arange(0,1 + tau,tau)
    print("L = ", L, "T = ", T)
    # print(len(t))
    # h = mpf(1/(L-1))
    # t, tau = np.linspace(0, 1, T , retstep=True)
    # tau = mpf(1/(T-1))
    u_next = lambda curr,next1,next2,next3,tau,h: mpf(curr) + mpf((tau)/(mpf(mpf('3.0')*h)))*(mpf(mpf('2.0')*next3)-mpf(mpf('9.0')*next2)+mpf(mpf('18.0')*next1)-mpf(mpf('11.0')*curr))+mpf((mpf(mpf('2.0')*tau**2))/(h**2))*(-next3+mpf(mpf('4.0')*next2)-mpf(mpf('5.0')*next1)+mpf(mpf('2.0')*curr)) + mpf(mpf(mpf('4.0')*tau**2))/(mpf(mpf('3.0')*h**2)) * mpf(next3-mpf(mpf('3.0')*next2)+mpf(mpf('3.0')*next1)-curr)
    u_l0 = lambda x: mp.cos(x)
    u_Ln = lambda t: mp.cos(1.0+mpf(2.0*t))
    u = [([0]*(len(x))) for i in range(len(t))]; ux0 = []; uLn = []
    # print("L = ", L)
    # print("L = ", L, "T = ", T)
    for i in range(0,len(x)): 
            u[0][i] = u_l0(x[i])
    for k in range(1,len(t)):
            u[k][len(x)-1] = u_Ln(t[k])
            # print(u[k][len(x)-1])
    for j in range(0,len(t)-1):
        u[j+1][len(x)-2]= mpf(mp.cos(mpf('1.0')+mpf(2.0*t[j+1])) + h*mp.sin(1.0+mpf(2.0*t[j+1]))-mpf(h**2/2.0)*mp.cos(1.0+mpf(2.0*t[j+1]))-mpf((h**3)/6.0) * mp.sin(1.0+mpf(2.0*t[j+1])))
        u[j+1][len(x)-3] = mpf(mp.cos(1.0+mpf(2.0*t[j+1])) + 2*h*mp.sin(1.0+mpf(2.0*t[j+1]))-mpf(2.0*(h**2))*mp.cos(1.0+mpf(2.0*t[j+1]))-mpf(4.0/3.0*(h**3)) * mp.sin(1.0+mpf(2.0*t[j+1])))
        for i in range(len(x)-4,-1,-1):
            u[j+1][i] = u_next(u[j][i],u[j][i+1],u[j][i+2],u[j][i+3],tau,h)
    solutions = []
    for i in range(len(x)):
        solutions.append(float(u[len(t)-1][i]))
        # print(u[i][len(t)-1])
    print("Число куранта = {}".format(tau/h))
    # print("h = {}".format(h), "tau = {}".format(tau), "Число куранта = {}".format(tau/h))
    return solutions[::(koef)]

* ### Таблица результатов

In [577]:
x,u1 = analytic_points(1.0)
koef = 8
u2 = solve(koef,0.5)
delta1 = []; 
for i in range(len(u1)):
    delta1.append(abs(u1[i] - u2[i]))
pd.options.display.float_format = "{:.4e}".format
print("Максимальная норма ошибки =", max(delta1))
data = pd.DataFrame({"x": x, "u аналит.": u1, "u числ. ": u2 ,"u числ.- u аналит.": delta1})
display(data)

L =  81 T =  161
Число куранта = 0.5
Максимальная норма ошибки = 2.3029994948020605e-05


,x,u аналит.,u числ.,u числ.- u аналит.
0,0.0000e+00,-4.1615e-01,-4.1612e-01,2.3030e-05
1,1.0000e-01,-5.0485e-01,-5.0483e-01,1.9607e-05
2,2.0000e-01,-5.8850e-01,-5.8848e-01,1.6254e-05
3,3.0000e-01,-6.6628e-01,-6.6626e-01,1.3047e-05
4,4.0000e-01,-7.3739e-01,-7.3738e-01,1.0058e-05
5,5.0000e-01,-8.0114e-01,-8.0114e-01,7.3549e-06
6,6.0000e-01,-8.5689e-01,-8.5688e-01,4.9965e-06
7,7.0000e-01,-9.0407e-01,-9.0407e-01,3.0355e-06
8,8.0000e-01,-9.4222e-01,-9.4222e-01,1.5156e-06
9,9.0000e-01,-9.7096e-01,-9.7096e-01,4.7163e-07


* ### Исследование на устойчивость

In [597]:
x,u1 = analytic_points(1)
i = 0; 
kurant = [0.2,0.3,0.4,0.5,0.6,0.7, 0.8, 0.9, 1]

In [598]:
L = [201 for i in range(len(kurant))]; deltas = []
for k in kurant:
    delta1 = []
    u2 = solve(20,k)
    for i in range(len(u1)):
        delta1.append(abs(u1[i] - u2[i]))
    deltas.append(max(delta1))
# pd.options.display.float_format = "{:.4e}".format
data = pd.DataFrame({"L": L, "Число Куранта": kurant,"Ошибка": deltas})
display(data)

L =  201 T =  1001
Число куранта = 0.2
L =  201 T =  667
Число куранта = 0.3003003003003003
L =  201 T =  501
Число куранта = 0.4
L =  201 T =  401
Число куранта = 0.5
L =  201 T =  334
Число куранта = 0.6006006006006006
L =  201 T =  286
Число куранта = 0.7017543859649123
L =  201 T =  251
Число куранта = 0.8
L =  201 T =  223
Число куранта = 0.9009009009009009
L =  201 T =  201
Число куранта = 1.0


,L,Число Куранта,Ошибка
0,201,2.0000e-01,2.3857e-06
1,201,3.0000e-01,3.1402e-06
2,201,4.0000e-01,3.5923e-06
3,201,5.0000e-01,3.7473e-06
4,201,6.0000e-01,1.7099e+40
5,201,7.0000e-01,1.0970e+40
6,201,8.0000e-01,2.6718e+03
7,201,9.0000e-01,1.3436e-06
8,201,1.0000e+00,4.0535e-10


In [599]:
L = [501 for i in range(len(kurant))]; deltas = []
for k in kurant:
    delta1 = []
    u2 = solve(50,k)
    for i in range(len(u1)):
        delta1.append(abs(u1[i] - u2[i]))
    deltas.append(max(delta1))
# pd.options.display.float_format = "{:.4e}".format
data = pd.DataFrame({"L": L, "Число Куранта": kurant,"Ошибка": deltas})
display(data)

L =  501 T =  2501
Число куранта = 0.2
L =  501 T =  1667
Число куранта = 0.30012004801920765
L =  501 T =  1251
Число куранта = 0.4
L =  501 T =  1001
Число куранта = 0.5
L =  501 T =  834
Число куранта = 0.6002400960384153
L =  501 T =  715
Число куранта = 0.70028011204481795
L =  501 T =  626
Число куранта = 0.8
L =  501 T =  556
Число куранта = 0.9009009009009009
L =  501 T =  501
Число куранта = 1.0


,L,Число Куранта,Ошибка
0,501,2.0000e-01,3.8545e-07
1,501,3.0000e-01,1.0908e-03
2,501,4.0000e-01,5.7907e-07
3,501,5.0000e-01,1.4004e-06
4,501,6.0000e-01,8.0388e+115
5,501,7.0000e-01,2.6581e+115
6,501,8.0000e-01,8.9581e+21
7,501,9.0000e-01,2.1588e-07
8,501,1.0000e+00,1.0365e-11


In [601]:
L = [801 for i in range(len(kurant))]; deltas = []
for k in kurant:
    delta1 = []
    u2 = solve(80,k)
    for i in range(len(u1)):
        delta1.append(abs(u1[i] - u2[i]))
    deltas.append(max(delta1))
# pd.options.display.float_format = "{:.4e}".format
data = pd.DataFrame({"L": L, "Число Куранта": kurant,"Ошибка": deltas})
display(data)

L =  801 T =  4001
Число куранта = 0.2
L =  801 T =  2667
Число куранта = 0.30007501875468868
L =  801 T =  2001
Число куранта = 0.4
L =  801 T =  1601
Число куранта = 0.5
L =  801 T =  1334
Число куранта = 0.60015003750937736
L =  801 T =  1143
Число куранта = 0.70052539404553416
L =  801 T =  1001
Число куранта = 0.8
L =  801 T =  889
Число куранта = 0.90090090090090088
L =  801 T =  801
Число куранта = 1.0


,L,Число Куранта,Ошибка
0,801,2.0000e-01,1.5094e-07
1,801,3.0000e-01,1.9824e-07
2,801,4.0000e-01,2.2662e-07
3,801,5.0000e-01,5.7953e-03
4,801,6.0000e-01,5.9559e+192
5,801,7.0000e-01,1.1940e+191
6,801,8.0000e-01,1.4375e+41
7,801,9.0000e-01,8.4416e-08
8,801,1.0000e+00,1.5812e-12
